In [1]:
%%configure -f
{"conf":
 {"spark.driver.cores": "2",
  "spark.driver.memory": "3g",
  "spark.executor.cores": "2",
  "spark.executor.memory": "3g",
  "spark.dynamicAllocation.enabled": "true",
  "spark.dynamicAllocation.minExecutors" : "3"
}
}

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as sf
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test')\
.enableHiveSupport()\
.config("hive.exec.dynamic.partition.mode", "nonstrict") \
.config("hive.exec.dynamic.partition", "true") \
.getOrCreate()

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

In [4]:

#--------------------------------------------------------------------------
#Get yesterday's date and previous month in correct format used in partitioning
#--------------------------------------------------------------------------
yesterday = datetime.date.today() - datetime.timedelta(days=1)

first_day_of_this_month = yesterday.replace(day=1)
last_day_of_previous_month = first_day_of_this_month - datetime.timedelta(days=1)
month = last_day_of_previous_month.strftime("%Y%m")
yesterday = yesterday.strftime("%Y%m%d")


#Data settings
attempts_weight = 1.4 #The bigger this variable is, the more weight is given to attempts versus usage when defining weight for application per user/cell
wished_retransmissions = 0.99 #Wished retransmission rate
attempts_ratio = 0.1 #Filters applications that are responsible for  < attempts_ratio

#Voice Settings
a1 = 0.5
a2 = 0.25
a3 = 0.25
              
UserWishedSpRetSuccRate = 0.99
UserWishedSpAccSuccRate = 0.99
UserWishedAvgCallSetupTime = 2.8

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:

cdp = spark.read.option("header",True).option("inferSchema",True)\
   .csv("hdfs:///user/sondkara/celldataperformance/sample_celldataperformance.csv")
cdp.printSchema()
cdp.registerTempTable('cellDataPerformance')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- basestationinformationid: string (nullable = true)
 |-- gn105_rtt_packets_1: integer (nullable = true)
 |-- gn106_rtt_packets_2: integer (nullable = true)
 |-- gn107_rtt_packets_3: integer (nullable = true)
 |-- gn108_rtt_packets_4: integer (nullable = true)
 |-- gn109_rtt_packets_5: integer (nullable = true)
 |-- gn110_rtt_packets_6: integer (nullable = true)
 |-- gn111_rtt_packets_7: integer (nullable = true)
 |-- gn112_rtt_packets_8: integer (nullable = true)
 |-- gn113_rtt_packets_9: integer (nullable = true)
 |-- gn114_rtt_packets_10: integer (nullable = true)
 |-- gn22_bytes_up: integer (nullable = true)
 |-- gn23_bytes_down: integer (nullable = true)
 |-- gn24_bytes_total: integer (nullable = true)
 |-- gn304_throughputbytesdown11: integer (nullable = true)
 |-- gn305_throughputbytesdown12: integer (nullable = true)
 |-- gn306_throughputtimedown11: integer (nullable = true)
 |-- gn307_throughputtimedown12: integer (nullable = true)
 |-- gn65_throughput_bytes_up_1: inte

In [16]:
cdp.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------+---------------+----------------+---------------------------+---------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+---------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------------------+----------------------------+----------------------------+--------------------------

In [6]:
import pandas as pd
weights_dict = {'application_group': ['App_Stores','Network_Operation','Test_applications','Uncategorised','email', 'file_transfer','gaming','messaging','peer_to_peer','security','streaming_applications','updates','voice_over_ip','web_applications'], 
                'b1': [0.2,0.2,0,0.1,0.2,0.2,0.15,0.1,0.2,0.2,0.2,0.2,0.1,0.1],
                'b2': [0.2,0,0,0,0,0.15,0,0,0,0,0.2,0.2,0,0.15],
                'b3': [0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                'b4': [0.4,0.3,0.4,0.4,0.3,0.3,0.4,0.3,0.45,0.3,0.3,0.25,0.3,0.3],
                'b5': [0,0.2,0.3,0.2,0.3,0.2,0.1,0.3,0.2,0.3,0.2,0.2,0.3,0.15],
                'b6': [0.2,0.3,0.3,0.3,0.2,0.15,0.35,0.3,0.15,0.2,0.1,0.15,0.3,0.3]}
#Weights for data
weights_data = spark.createDataFrame(pd.DataFrame(weights_dict))
weights_data.registerTempTable('weights_data')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
for par_msisdn in range(0,10):
  qry = """
SELECT LogicalResourceId,
       applicationName,
       first(mobelideviceresourcename) as MobileDeviceResourceName,
       first(par_msisdn) as par_msisdn,
       first(category) as category,
                     
       --Calculate totals for time down
       (NVL(SUM(gn95_throughput_time_down_1),0) + NVL(SUM(lte94_throughput_time_down_1),0) +
       NVL(SUM(gn96_throughput_time_down_2),0) + NVL(SUM(lte95_throughput_time_down_2),0) + 
       NVL(SUM(gn97_throughput_time_down_3),0) + NVL(SUM(lte96_throughput_time_down_3),0) +               
       NVL(SUM(gn98_throughput_time_down_4),0) + NVL(SUM(lte97_throughput_time_down_4),0) +               
       NVL(SUM(gn99_throughput_time_down_5),0) + NVL(SUM(lte98_throughput_time_down_5),0) +               
       NVL(SUM(gn100_throughput_time_down_6),0) + NVL(SUM(lte99_throughput_time_down_6),0)+               
       NVL(SUM(gn101_throughput_time_down_7),0) + NVL(SUM(lte100_throughput_time_down_7),0)+              
       NVL(SUM(gn102_throughput_time_down_8),0) + NVL(SUM(lte101_throughput_time_down_8),0)+              
       NVL(SUM(gn103_throughput_time_down_9),0) + NVL(SUM(lte102_throughput_time_down_9),0)+
       NVL(SUM(gn104_throughput_time_down_10),0) + NVL(SUM(lte103_throughput_time_down_10),0)+           
       NVL(SUM(gn306_throughputtimedown11),0) + NVL(SUM(lte306_throughputtimedown11),0) +                
       NVL(SUM(gn307_throughputtimedown12),0) + NVL(SUM(lte307_throughputtimedown12),0)) AS total_time_down, 
                     
       --Calculate totals for time upload
       (NVL(SUM(gn75_throughput_time_up_1),0) + NVL(SUM(lte74_throughput_time_up_1),0) +
       NVL(SUM(gn76_throughput_time_up_2),0) + NVL(SUM(lte75_throughput_time_up_2 ),0)+
       NVL(SUM(gn77_throughput_time_up_3),0) + NVL(SUM(lte76_throughput_time_up_3 ),0)+
       NVL(SUM(gn78_throughput_time_up_4),0) + NVL(SUM(lte77_throughput_time_up_4 ),0)+
       NVL(SUM(gn79_throughput_time_up_5),0) + NVL(SUM(lte78_throughput_time_up_5 ),0)+
       NVL(SUM(gn80_throughput_time_up_6),0) + NVL(SUM(lte79_throughput_time_up_6 ),0)+
       NVL(SUM(gn81_throughput_time_up_7),0) + NVL(SUM(lte80_throughput_time_up_7 ),0)+
       NVL(SUM(gn82_throughput_time_up_8),0) + NVL(SUM(lte81_throughput_time_up_8 ),0)+
       NVL(SUM(gn83_throughput_time_up_9),0) + NVL(SUM(lte82_throughput_time_up_9 ),0)+
       NVL(SUM(gn84_throughput_time_up_10),0)+ NVL(SUM(lte83_throughput_time_up_10),0)) AS total_time_up,
                     
       --Calculate totals for bytes down
       (NVL(SUM(gn85_throughput_bytes_down_1),0) + NVL(SUM(lte84_throughput_bytes_down_1 ),0)+
        NVL(SUM(gn86_throughput_bytes_down_2),0) + NVL(SUM(lte85_throughput_bytes_down_2 ),0)+
        NVL(SUM(gn87_throughput_bytes_down_3),0) + NVL(SUM(lte86_throughput_bytes_down_3 ),0)+
        NVL(SUM(gn88_throughput_bytes_down_4),0) + NVL(SUM(lte87_throughput_bytes_down_4 ),0)+
        NVL(SUM(gn89_throughput_bytes_down_5),0) + NVL(SUM(lte88_throughput_bytes_down_5 ),0)+
        NVL(SUM(gn90_throughput_bytes_down_6),0) + NVL(SUM(lte89_throughput_bytes_down_6 ),0)+
        NVL(SUM(gn91_throughput_bytes_down_7),0) + NVL(SUM(lte90_throughput_bytes_down_7 ),0)+
        NVL(SUM(gn92_throughput_bytes_down_8),0) + NVL(SUM(lte91_throughput_bytes_down_8 ),0)+
        NVL(SUM(gn93_throughput_bytes_down_9),0) + NVL(SUM(lte92_throughput_bytes_down_9 ),0)+
        NVL(SUM(gn94_throughput_bytes_down_10),0)+ NVL(SUM(lte93_throughput_bytes_down_10),0)+
        NVL(SUM(gn304_throughputbytesdown11),0)  + NVL(SUM(lte304_throughputbytesdown11),0)  +
        NVL(SUM(gn305_throughputbytesdown12),0)  + NVL(SUM(lte305_throughputbytesdown12),0)) AS total_bytes_down,
                      
       --Calculate totals for bytes up
       (NVL(SUM(gn65_throughput_bytes_up_1),0) + NVL(SUM(lte64_throughput_bytes_up_1),0) +
       NVL(SUM(gn66_throughput_bytes_up_2),0) + NVL(SUM(lte65_throughput_bytes_up_2),0) +
       NVL(SUM(gn67_throughput_bytes_up_3),0) + NVL(SUM(lte66_throughput_bytes_up_3),0) +
       NVL(SUM(gn68_throughput_bytes_up_4),0) + NVL(SUM(lte67_throughput_bytes_up_4),0) +
       NVL(SUM(gn69_throughput_bytes_up_5),0) + NVL(SUM(lte68_throughput_bytes_up_5),0) +
       NVL(SUM(gn70_throughput_bytes_up_6),0) + NVL(SUM(lte69_throughput_bytes_up_6),0) +
       NVL(SUM(gn71_throughput_bytes_up_7),0) + NVL(SUM(lte70_throughput_bytes_up_7),0) +
       NVL(SUM(gn72_throughput_bytes_up_8),0) + NVL(SUM(lte71_throughput_bytes_up_8),0) +
       NVL(SUM(gn73_throughput_bytes_up_9),0) + NVL(SUM(lte72_throughput_bytes_up_9),0) +
       NVL(SUM(gn74_throughput_bytes_up_10),0)+ NVL(SUM(lte73_throughput_bytes_up_10),0)) as total_bytes_up,
                      
       --Calculate totals for latency
       (NVL(SUM(gn105_rtt_packets_1),0) + NVL(SUM(lte104_rtt_packets_1),0) +
       NVL(SUM(gn106_rtt_packets_2),0) + NVL(SUM(lte105_rtt_packets_2),0) +
       NVL(SUM(gn107_rtt_packets_3),0) + NVL(SUM(lte106_rtt_packets_3),0) +
       NVL(SUM(gn108_rtt_packets_4),0) + NVL(SUM(lte107_rtt_packets_4),0) +
       NVL(SUM(gn109_rtt_packets_5),0) + NVL(SUM(lte108_rtt_packets_5),0) +
       NVL(SUM(gn110_rtt_packets_6),0) + NVL(SUM(lte109_rtt_packets_6),0) +
       NVL(SUM(gn111_rtt_packets_7),0) + NVL(SUM(lte110_rtt_packets_7),0) +
       NVL(SUM(gn112_rtt_packets_8),0) + NVL(SUM(lte111_rtt_packets_8),0) +
       NVL(SUM(gn113_rtt_packets_9),0) + NVL(SUM(lte112_rtt_packets_9),0) +
       NVL(SUM(gn114_rtt_packets_10),0)+ NVL(SUM(lte113_rtt_packets_10),0)) as total_latency,
                     
       --Calculate total attempts and total packets
       NVL(SUM(gn27_total_packets),0) + NVL(SUM(lte26_total_packets),0) as total_packets,
       NVL(SUM(gn30_retransmitted_total_packets),0) + NVL(SUM(lte29_retransmitted_total_packets),0) as retransmitted_total_packets,
       NVL(SUM(lte19_total),0) + NVL(SUM(gn20_total),0) as total_attempts,
                     
       -- Calculate throughput_time_down which correlates with b2
       NVL(SUM(gn95_throughput_time_down_1),0) + NVL(SUM(lte94_throughput_time_down_1),0) as throughput_time_down_1,
       NVL(SUM(gn96_throughput_time_down_2),0) + NVL(SUM(lte95_throughput_time_down_2),0) as throughput_time_down_2,
       NVL(SUM(gn97_throughput_time_down_3),0) + NVL(SUM(lte96_throughput_time_down_3),0) as throughput_time_down_3,
       NVL(SUM(gn98_throughput_time_down_4),0) + NVL(SUM(lte97_throughput_time_down_4),0) as throughput_time_down_4,
       NVL(SUM(gn99_throughput_time_down_5),0) + NVL(SUM(lte98_throughput_time_down_5),0) as throughput_time_down_5,
       NVL(SUM(gn100_throughput_time_down_6),0) + NVL(SUM(lte99_throughput_time_down_6),0) as throughput_time_down_6,
       NVL(SUM(gn101_throughput_time_down_7),0) + NVL(SUM(lte100_throughput_time_down_7),0) as throughput_time_down_7,
       NVL(SUM(gn102_throughput_time_down_8),0) + NVL(SUM(lte101_throughput_time_down_8),0) as throughput_time_down_8,
       NVL(SUM(gn103_throughput_time_down_9),0) + NVL(SUM(lte102_throughput_time_down_9),0) as throughput_time_down_9,
       NVL(SUM(gn104_throughput_time_down_10),0) + NVL(SUM(lte103_throughput_time_down_10),0) as throughput_time_down_10,
       NVL(SUM(gn306_throughputtimedown11),0) + NVL(SUM(lte306_throughputtimedown11),0) as throughput_time_down_11,
       NVL(SUM(gn307_throughputtimedown12),0) + NVL(SUM(lte307_throughputtimedown12),0) as throughput_time_down_12,
                     
       -- Calculate throughput_time_up which correlates with b3
       NVL(SUM(gn75_throughput_time_up_1),0) + NVL(SUM(lte74_throughput_time_up_1),0) as throughput_time_up_1,
       NVL(SUM(gn76_throughput_time_up_2),0) + NVL(SUM(lte75_throughput_time_up_2 ),0) as throughput_time_up_2,
       NVL(SUM(gn77_throughput_time_up_3),0) + NVL(SUM(lte76_throughput_time_up_3 ),0) as throughput_time_up_3,
       NVL(SUM(gn78_throughput_time_up_4),0) + NVL(SUM(lte77_throughput_time_up_4 ),0) as throughput_time_up_4,
       NVL(SUM(gn79_throughput_time_up_5),0) + NVL(SUM(lte78_throughput_time_up_5 ),0) as throughput_time_up_5,
       NVL(SUM(gn80_throughput_time_up_6),0) + NVL(SUM(lte79_throughput_time_up_6 ),0) as throughput_time_up_6,
       NVL(SUM(gn81_throughput_time_up_7),0) + NVL(SUM(lte80_throughput_time_up_7 ),0) as throughput_time_up_7,
       NVL(SUM(gn82_throughput_time_up_8),0) + NVL(SUM(lte81_throughput_time_up_8 ),0) as throughput_time_up_8,
       NVL(SUM(gn83_throughput_time_up_9),0) + NVL(SUM(lte82_throughput_time_up_9 ),0) as throughput_time_up_9,
       NVL(SUM(gn84_throughput_time_up_10),0)+ NVL(SUM(lte83_throughput_time_up_10),0) as throughput_time_up_10,
                     
                     
       -- Calculate throughput_buytes_down which correlates with b4
       NVL(SUM(gn85_throughput_bytes_down_1),0) + NVL(SUM(lte84_throughput_bytes_down_1 ),0) as throughput_bytes_down_1,
       NVL(SUM(gn86_throughput_bytes_down_2),0) + NVL(SUM(lte85_throughput_bytes_down_2 ),0) as throughput_bytes_down_2,
       NVL(SUM(gn87_throughput_bytes_down_3),0) + NVL(SUM(lte86_throughput_bytes_down_3 ),0) as throughput_bytes_down_3,
       NVL(SUM(gn88_throughput_bytes_down_4),0) + NVL(SUM(lte87_throughput_bytes_down_4 ),0) as throughput_bytes_down_4,
       NVL(SUM(gn89_throughput_bytes_down_5),0) + NVL(SUM(lte88_throughput_bytes_down_5 ),0) as throughput_bytes_down_5,
       NVL(SUM(gn90_throughput_bytes_down_6),0) + NVL(SUM(lte89_throughput_bytes_down_6 ),0) as throughput_bytes_down_6,
       NVL(SUM(gn91_throughput_bytes_down_7),0) + NVL(SUM(lte90_throughput_bytes_down_7 ),0) as throughput_bytes_down_7,
       NVL(SUM(gn92_throughput_bytes_down_8),0) + NVL(SUM(lte91_throughput_bytes_down_8 ),0) as throughput_bytes_down_8,
       NVL(SUM(gn93_throughput_bytes_down_9),0) + NVL(SUM(lte92_throughput_bytes_down_9 ),0) as throughput_bytes_down_9,
       NVL(SUM(gn94_throughput_bytes_down_10),0)+ NVL(SUM(lte93_throughput_bytes_down_10),0) as throughput_bytes_down_10,
       NVL(SUM(gn304_throughputbytesdown11),0)  + NVL(SUM(lte304_throughputbytesdown11),0)   as throughput_bytes_down_11,
       NVL(SUM(gn305_throughputbytesdown12),0)  + NVL(SUM(lte305_throughputbytesdown12),0)   as throughput_bytes_down_12,


       -- Calculate throughput_buytes_up which correlates with b5
       NVL(SUM(gn65_throughput_bytes_up_1),0) + NVL(SUM(lte64_throughput_bytes_up_1),0) as throughput_bytes_up_1,
       NVL(SUM(gn66_throughput_bytes_up_2),0) + NVL(SUM(lte65_throughput_bytes_up_2),0) as throughput_bytes_up_2,
       NVL(SUM(gn67_throughput_bytes_up_3),0) + NVL(SUM(lte66_throughput_bytes_up_3),0) as throughput_bytes_up_3,
       NVL(SUM(gn68_throughput_bytes_up_4),0) + NVL(SUM(lte67_throughput_bytes_up_4),0) as throughput_bytes_up_4,
       NVL(SUM(gn69_throughput_bytes_up_5),0) + NVL(SUM(lte68_throughput_bytes_up_5),0) as throughput_bytes_up_5,
       NVL(SUM(gn70_throughput_bytes_up_6),0) + NVL(SUM(lte69_throughput_bytes_up_6),0) as throughput_bytes_up_6,
       NVL(SUM(gn71_throughput_bytes_up_7),0) + NVL(SUM(lte70_throughput_bytes_up_7),0) as throughput_bytes_up_7,
       NVL(SUM(gn72_throughput_bytes_up_8),0) + NVL(SUM(lte71_throughput_bytes_up_8),0) as throughput_bytes_up_8,
       NVL(SUM(gn73_throughput_bytes_up_9),0) + NVL(SUM(lte72_throughput_bytes_up_9),0) as throughput_bytes_up_9,
       NVL(SUM(gn74_throughput_bytes_up_10),0)+ NVL(SUM(lte73_throughput_bytes_up_10),0) as throughput_bytes_up_10,
       
       
       --Calculate rtt_packets which correlates with b6
       NVL(SUM(gn105_rtt_packets_1),0) + NVL(SUM(lte104_rtt_packets_1),0) as rtt_packets_1,
       NVL(SUM(gn106_rtt_packets_2),0) + NVL(SUM(lte105_rtt_packets_2),0) as rtt_packets_2,
       NVL(SUM(gn107_rtt_packets_3),0) + NVL(SUM(lte106_rtt_packets_3),0) as rtt_packets_3,
       NVL(SUM(gn108_rtt_packets_4),0) + NVL(SUM(lte107_rtt_packets_4),0) as rtt_packets_4,
       NVL(SUM(gn109_rtt_packets_5),0) + NVL(SUM(lte108_rtt_packets_5),0) as rtt_packets_5,
       NVL(SUM(gn110_rtt_packets_6),0) + NVL(SUM(lte109_rtt_packets_6),0) as rtt_packets_6,
       NVL(SUM(gn111_rtt_packets_7),0) + NVL(SUM(lte110_rtt_packets_7),0) as rtt_packets_7,
       NVL(SUM(gn112_rtt_packets_8),0) + NVL(SUM(lte111_rtt_packets_8),0) as rtt_packets_8,
       NVL(SUM(gn113_rtt_packets_9),0) + NVL(SUM(lte112_rtt_packets_9),0) as rtt_packets_9,
       NVL(SUM(gn114_rtt_packets_10),0)+ NVL(SUM(lte113_rtt_packets_10),0) as rtt_packets_10
                     
FROM test1.celldataperformance_csi
WHERE par_dt = '{date}' AND
      par_msisdn = {par_msisdn} AND
      applicationName not in('ALL', 'VoLTE') AND
      application_protocol not like 'quic' AND 
      (
      ((gn85_throughput_bytes_down_1 >=500000) OR
      (gn86_throughput_bytes_down_2 >=500000) OR 
      (gn87_throughput_bytes_down_3 >=500000) OR
      (gn88_throughput_bytes_down_4 >=500000)OR
      (gn89_throughput_bytes_down_5 >=500000)OR
      (gn90_throughput_bytes_down_6 >=500000)OR
      (gn91_throughput_bytes_down_7 >=500000)OR
      (gn92_throughput_bytes_down_8 >=500000)OR
      (gn93_throughput_bytes_down_9 >=500000)OR
      (gn94_throughput_bytes_down_10 >=500000)OR
      (lte84_throughput_bytes_down_1 >=500000)OR 
      (lte85_throughput_bytes_down_2 >=500000)OR 
      (lte86_throughput_bytes_down_3 >=500000)OR 
      (lte87_throughput_bytes_down_4 >=500000)OR
      (lte88_throughput_bytes_down_5 >=500000)OR 
      (lte89_throughput_bytes_down_6 >=500000)OR 
      (lte90_throughput_bytes_down_7 >=500000)OR 
      (lte91_throughput_bytes_down_8 >=500000)OR
      (lte92_throughput_bytes_down_9 >=500000)OR 
      (lte93_throughput_bytes_down_10 >=500000) OR
      (gn304_throughputbytesdown11 >= 500000) OR
      (gn305_throughputbytesdown12 >= 500000) OR
      (lte304_throughputbytesdown11 >= 500000) OR
      (lte305_throughputbytesdown12 >= 500000) ) OR 
      ((lte64_throughput_bytes_up_1 >=50000)OR 
      (lte65_throughput_bytes_up_2 >=50000)OR 
      (lte66_throughput_bytes_up_3 >=50000)OR 
      (lte67_throughput_bytes_up_4 >=50000)OR
      (lte68_throughput_bytes_up_5 >=50000)OR 
      (lte69_throughput_bytes_up_6 >=50000)OR 
      (lte70_throughput_bytes_up_7 >=50000)OR 
      (lte71_throughput_bytes_up_8 >=50000)OR
      (lte72_throughput_bytes_up_9 >=50000)OR 
      (lte73_throughput_bytes_up_10 >=50000) OR
      (gn65_throughput_bytes_up_1 >= 50000) OR
      (gn66_throughput_bytes_up_2 >= 50000) OR
      (gn67_throughput_bytes_up_3 >= 50000) OR
      (gn68_throughput_bytes_up_4 >= 50000) OR
      (gn69_throughput_bytes_up_5 >= 50000) OR
      (gn70_throughput_bytes_up_6 >= 50000) OR
      (gn71_throughput_bytes_up_7 >= 50000) OR
      (gn72_throughput_bytes_up_8 >= 50000) OR
      (gn73_throughput_bytes_up_9 >= 50000) OR
      (gn74_throughput_bytes_up_10 >= 50000)))                                                  
GROUP BY LogicalResourceId, applicationName
""".format(date=yesterday, month=month, par_msisdn = par_msisdn)

  initial_aggregation = spark.sql(qry)
  initial_aggregation.registerTempTable('initial_aggregation')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
initial_aggregation.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------------+------------------------+----------+--------+---------------+-------------+----------------+--------------+-------------+-------------+---------------------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------------+------------------------+------------------------+------

In [8]:
qry = """
SELECT
      LogicalResourceId,
      MobileDeviceResourceName,
      applicationName,
      par_msisdn,
      category,
      total_attempts,
      (total_bytes_down + total_bytes_up) as total_volume,
            
      (total_attempts/SUM(total_attempts) OVER (PARTITION by LogicalResourceId)) total_attempts_ratio,

      --Calculate b1 kpi fraction for total transmitted packets
      (1 - (retransmitted_total_packets/total_packets))/{wished_retransmissions} AS b1_kpi_fraction,
                     
      -- Calculates percent for download bytes down
      ((throughput_bytes_down_1/total_bytes_down * 1) +
      (throughput_bytes_down_2/total_bytes_down * 2) +
      (throughput_bytes_down_3/total_bytes_down * 3) +
      (throughput_bytes_down_4/total_bytes_down * 4) +
      (throughput_bytes_down_5/total_bytes_down * 5) +
      (throughput_bytes_down_6/total_bytes_down * 6) +
      (throughput_bytes_down_7/total_bytes_down * 7) +
      (throughput_bytes_down_8/total_bytes_down * 8) +
      (throughput_bytes_down_9/total_bytes_down * 9) +
      (throughput_bytes_down_10/total_bytes_down * 10) +
      (throughput_bytes_down_11/total_bytes_down * 11) +
      (throughput_bytes_down_12/total_bytes_down * 12)) as b4_kpi_nominator,
      
      --Calculate percent for throughput bytes up
      ((throughput_bytes_up_1/total_bytes_up * 1)  +
      (throughput_bytes_up_2/total_bytes_up * 2)  +
      (throughput_bytes_up_3/total_bytes_up * 3)  +
      (throughput_bytes_up_4/total_bytes_up * 4)  +
      (throughput_bytes_up_5/total_bytes_up * 5)  +
      (throughput_bytes_up_6/total_bytes_up * 6)  +
      (throughput_bytes_up_7/total_bytes_up * 7)  +
      (throughput_bytes_up_8/total_bytes_up * 8)  +
      (throughput_bytes_up_9/total_bytes_up * 9)  +
      (throughput_bytes_up_10/total_bytes_up * 10)) as b5_kpi_nominator,
      
      --Calculate percent for throughput time up
      ((throughput_time_up_1/total_time_up * 1)   +
      (throughput_time_up_2/total_time_up * 2)   +
      (throughput_time_up_3/total_time_up * 3)   +
      (throughput_time_up_4/total_time_up * 4)   +
      (throughput_time_up_5/total_time_up * 5)   +
      (throughput_time_up_6/total_time_up * 6)   +
      (throughput_time_up_7/total_time_up * 7)   +
      (throughput_time_up_8/total_time_up * 8)   +
      (throughput_time_up_9/total_time_up * 9)   +
      (throughput_time_up_10/total_time_up * 10)) as b3_kpi_nominator,
      
      --Calculates percent for throughput time down
      ((throughput_time_down_1/total_time_down * 1) +
      (throughput_time_down_2/total_time_down * 2) +
      (throughput_time_down_3/total_time_down * 3) +
      (throughput_time_down_4/total_time_down * 4) +
      (throughput_time_down_5/total_time_down * 5) +
      (throughput_time_down_6/total_time_down * 6) +
      (throughput_time_down_7/total_time_down * 7) +
      (throughput_time_down_8/total_time_down * 8) +
      (throughput_time_down_9/total_time_down * 9) +
      (throughput_time_down_10/total_time_down * 10) +
      (throughput_time_down_11/total_time_down * 11) +
      (throughput_time_down_12/total_time_down * 12)) as b2_kpi_nominator,
      
      --Calculate percent for each rtt column   
      ((rtt_packets_1/total_latency * 10) +
      (rtt_packets_2/total_latency * 9) +
      (rtt_packets_3/total_latency * 8) +
      (rtt_packets_4/total_latency * 7) +
      (rtt_packets_5/total_latency * 6) +
      (rtt_packets_6/total_latency * 5) +
      (rtt_packets_7/total_latency * 4) +
      (rtt_packets_8/total_latency * 3) +
      (rtt_packets_9/total_latency * 2) +
      (rtt_packets_10/total_latency * 1)) as b6_kpi_nominator
      
FROM initial_aggregation""".format(wished_retransmissions = wished_retransmissions)

nominators = spark.sql(qry)
nominators.registerTempTable('nominators')


  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
nominators.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------------+---------------+----------+--------+--------------+------------+--------------------+---------------+----------------+----------------+----------------+----------------+----------------+
|LogicalResourceId|MobileDeviceResourceName|applicationName|par_msisdn|category|total_attempts|total_volume|total_attempts_ratio|b1_kpi_fraction|b4_kpi_nominator|b5_kpi_nominator|b3_kpi_nominator|b2_kpi_nominator|b6_kpi_nominator|
+-----------------+------------------------+---------------+----------+--------+--------------+------------+--------------------+---------------+----------------+----------------+----------------+----------------+----------------+
+-----------------+------------------------+---------------+----------+--------+--------------+------------+--------------------+---------------+----------------+----------------+----------------+----------------+----------------+

In [9]:
qry = """
SELECT
      c.LogicalResourceId,
      c.MobileDeviceResourceName,
      c.category,
      c.par_msisdn,
      denominators.*,
      c.b1_kpi_fraction,
      c.b2_kpi_nominator,
      c.b3_kpi_nominator,
      b4_kpi_nominator,
      c.b5_kpi_nominator,
      c.b6_kpi_nominator,
      c.total_attempts,
      c.total_volume,
      weights_data.b1,
      weights_data.b2,
      weights_data.b3,
      weights_data.b4,
      weights_data.b5,
      weights_data.b6   
FROM nominators C
LEFT JOIN ( SELECT *
FROM test1.csi_scores_per_application_normalized WHERE par_dt like '{month}' ) denominators
ON denominators.applicationName = C.applicationName
LEFT JOIN (SELECT upper(application_group) application_group, b1,b2,b3,b4,b5,b6 FROM weights_data) weights_data     
ON weights_data.application_group = C.category """.format(month=month)

weights = spark.sql(qry)
weights.registerTempTable('weights')

qry = """
SELECT 
      LogicalResourceId, 
      applicationName,
      total_attempts,
      total_volume,
      MobileDeviceResourceName,
      par_msisdn,
                
      -- Case when customer/cell downloads but doesnt upload using this app
CASE WHEN (b5_kpi_nominator IS NULL) AND (b4_kpi_nominator IS NOT NULL) 
THEN ( ((b1 + (b3/4) + (b5/4) ) * b1_kpi_fraction) +((b2 + (b3/4) + (b5/4) ) * b2_kpi_nominator/avg_throughput_score_time_down) + ((b4 + (b3/4) + (b5/4) ) * b4_kpi_nominator/avg_throughput_score_bytes_down) +((b6 + (b3/4) + (b5/4) ) * b6_kpi_nominator/avg_rtt_packets_score))
      -- Case when customer/cell uploads but doesnt download using this app
WHEN (b5_kpi_nominator IS NOT NULL) AND (b4_kpi_nominator IS NULL) 
THEN( ((b1 + (b2/4) + (b4/4) ) * b1_kpi_fraction) +((b3 + (b2/4) + (b4/4) ) * b3_kpi_nominator/avg_throughput_score_time_up) + ((b5 + (b2/4) + (b4/4) ) * b5_kpi_nominator/avg_throughput_score_bytes_up) +((b6 + (b2/4) + (b4/4) ) * b6_kpi_nominator/avg_rtt_packets_score))
      --Case when customer is normally downloading/uploading using applicationName
ELSE ( (b1 * b1_kpi_fraction) + (b2 * b2_kpi_nominator/avg_throughput_score_time_down) + (b3 * b3_kpi_nominator/avg_throughput_score_time_up) +(b4 * b4_kpi_nominator/avg_throughput_score_bytes_down) +(b5 * b5_kpi_nominator/avg_throughput_score_bytes_up)  + (b6 * b6_kpi_nominator/avg_rtt_packets_score))
END AS appCSI
FROM weights"""

csi_components = spark.sql(qry)
csi_components.registerTempTable('csi_components')

qry = """
SELECT 
      LogicalResourceId,
      par_msisdn,
      MobileDeviceResourceName,         
      applicationName,
      appCSI,
      total_attempts,
      total_volume,
      (sqrt(ln(total_volume+1.1) * pow(total_attempts, {attempts_weight}) ) ) as app_usage
FROM csi_components""".format(attempts_weight = attempts_weight)
usage1 = spark.sql(qry)

#usage=usage.unionAll(usage1)

usage1.registerTempTable('usage')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
usage1.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+------------------------+---------------+------+--------------+------------+---------+
|LogicalResourceId|par_msisdn|MobileDeviceResourceName|applicationName|appCSI|total_attempts|total_volume|app_usage|
+-----------------+----------+------------------------+---------------+------+--------------+------------+---------+
+-----------------+----------+------------------------+---------------+------+--------------+------------+---------+

In [10]:
qry = """
SELECT 
  applicationName,
  avg(appCSI) AS average_of_appCSI,
  '{date}' AS par_dt
FROM usage
GROUP BY applicationName""".format(date=yesterday)

avg_appCSI = spark.sql(qry)
avg_appCSI.write.insertInto("test1.csi_app_daily_normalized", overwrite = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
avg_appCSI.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+------+
|applicationName|average_of_appCSI|par_dt|
+---------------+-----------------+------+
+---------------+-----------------+------+